# Batch Coactivity Analysis

## Notebook for preprocessing and analyzing coactivity spine data

In [1]:
import os
import numpy as np
from Lab_Analyses.Spine_Analysis.preprocess_spine_data import organize_dual_spine_data
from Lab_Analyses.Spine_Analysis.grouped_coactivity_analysis import grouped_coactivity_analysis
from Lab_Analyses.Spine_Analysis.coactivity_plasticity_analysis import Coactivity_Plasticity
from Lab_Analyses.Utilities.save_load_pickle import load_pickle

### Datasets
Specify mice which mice to be processed and analyzed

In [2]:
# Mouse ids
id_1 = "JW082"
id_2 = "JW083"
id_3 = "JW090"
id_4 = "JW092"
id_5 = "JW126"
id_6 = "JW129"
id_7 = "JW130"

mouse_list = [id_1, id_2, id_3, id_4, id_5, id_6, id_7]
#mouse_list = [id_4, id_6]

### Preprocessing
Preprocess spine datasets

In [3]:
# Do you want to reprocess the data
reprocess = False
channels = {"GluSnFr": "GreenCh", "Calcium": "RedCh"}
fov_type = "apical"

# Process data
if reprocess:
    for mouse in mouse_list:
        organize_dual_spine_data(mouse, channels, fov_type=fov_type, redetection=True, save=True, structural=True)

### Grouped Coactivity Analysis
Perform coactivity analysis across all mice and FOVs for a given set of imaging sessions

In [ ]:
# Set up parameters
## Save path
group_path = r'C:\Users\Jake\Desktop\Analyzed_data\grouped'
experiment_dir = "Dual_Spine_Imaging"
coactivity_dir = "Coactivity_Data"
coactivity_path = os.path.join(group_path, experiment_dir, coactivity_dir)

## Analysis parameters
reanalyze = True
save = True
days = ["Early", "Middle", "Late"]
#days = ["Middle"]
followup = True
zscore = False
volume_norm = True
fov_type = "apical"

coactivity_data = {}
for day in days:
    print(f"-{day}")
    coactivity = grouped_coactivity_analysis(
        mouse_list,
        day,
        followup=followup,
        fov_type=fov_type,
        activity_window=(-2,4),
        zscore=zscore,
        volume_norm=volume_norm,
        save=save,
        save_path=coactivity_path,
    )

    coactivity_data[day] = coactivity

-Early
---------------------------------
- Analyzing JW082
-- Local Coactivity Analysis


C:\Users\Jake\Documents\Anaconda3\envs\analysis_env\lib\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\users\jake\desktop\python_code\lab_analyses\Lab_Analyses\Spine_Analysis\distance_coactivity_rate_analysis.py:199: RuntimeWarning: Mean of empty slice
  binned_data.append(np.nanmean(data[idxs]))
C:\Users\Jake\Documents\Anaconda3\envs\analysis_env\lib\site-packages\scipy\stats\stats.py:275: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
c:\users\jake\desktop\python_code\lab_analyses\Lab_Analyses\Spine_Analysis\distance_coactivity_rate_analysis.py:182: RuntimeWarning: invalid value encountered in double_scalars
  coactivity_rate = event_freq / geo_mean
c:\users\jake\desktop\python_code\lab_analyses\Lab_Analyses\Spine_Analysis\spine_coactivity_utilities.py:461: RuntimeWarning: Mean of empty slice
 

-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coactivity


c:\users\jake\desktop\python_code\lab_analyses\Lab_Analyses\Utilities\activity_onset.py:43: RuntimeWarning: Mean of empty slice
  trace_amp = np.nanmean(trace[trace_peak - window : trace_peak + window])


--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score


...

-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coactivity
--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score
-- Movement-Related Activity Analysis
-- Assessing Movement Quality
-Middle
---------------------------------
- Analyzing JW082
-- Local Coactivity Analysis
-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coactivity
--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score
-- Movement-Related Activity Analysis
-- Assessing Movement Quality
---------------------------------
- Analyzing JW083
-- Local Coactivity Analysis
-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coactivity
--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score


c:\users\jake\desktop\python_code\lab_analyses\Lab_Analyses\Spine_Analysis\calculate_cluster_score.py:155: RuntimeWarning: Mean of empty slice
  avg_nearest_neighbor = np.nanmean(nearest_neighbor)
c:\users\jake\desktop\python_code\lab_analyses\Lab_Analyses\Spine_Analysis\calculate_cluster_score.py:156: RuntimeWarning: Mean of empty slice
  avg_num_coactive = np.nanmean(number_coactive)


-- Movement-Related Activity Analysis
-- Assessing Movement Quality
---------------------------------
- Analyzing JW090
-- Local Coactivity Analysis
-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coactivity
--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score
-- Movement-Related Activity Analysis
-- Assessing Movement Quality
-- Local Coactivity Analysis
-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coactivity
--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score
-- Movement-Related Activity Analysis
-- Assessing Movement Quality
---------------------------------
- Analyzing JW092
-- Local Coactivity Analysis
-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coactivity
--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score
-- Movement-Related Activity Analysis
-- Assessing Movement Quality
-- Local Coactivity Analysis
-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coacti

c:\users\jake\desktop\python_code\lab_analyses\Lab_Analyses\Utilities\data_utilities.py:168: RuntimeWarning: Mean of empty slice
  m = np.nanmean(value, axis=1)
C:\Users\Jake\Documents\Anaconda3\envs\analysis_env\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Jake\Documents\Anaconda3\envs\analysis_env\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Jake\Documents\Anaconda3\envs\analysis_env\lib\site-packages\numpy\core\_methods.py:251: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score
-- Movement-Related Activity Analysis
-- Assessing Movement Quality
---------------------------------
- Analyzing JW129
-- Local Coactivity Analysis
-- Spine-Dendrite Coactivity Analysis
--- Analyzing Absolute Coactivity
--- Analyzing Distance-Dependence Coactivity
--- Calculating Cluster Score
-- Movement-Related Activity Analysis


In [ ]:
group_path = r'C:\Users\Jake\Desktop\Analyzed_data\grouped'
experiment_dir = "Dual_Spine_Imaging"
coactivity_dir = "Coactivity_Data"
coactivity_path = os.path.join(group_path, experiment_dir, coactivity_dir)

load_coactivity = True
if load_coactivity:
    names = ["Early_dFoF_norm_spine_coactivity_data", "Middle_dFoF_norm_spine_coactivity_data", "Late_dFoF_norm_spine_coactivity_data"]
    data = load_pickle(names, coactivity_path)
    days = ["Early", "Middle", "Late"]
    coactivity_data = dict(zip(days, data))

### Coactivity Plasticity Analysis
Perform analysis to relate subsequent plasticity to coactivity 

In [ ]:
## Save path
group_path = r'C:\Users\Jake\Desktop\Analyzed_data\grouped'
experiment_dir = "Dual_Spine_Imaging"
plasticity_dir = "Coactivity_Plasticity"
plasticity_path = os.path.join(group_path, experiment_dir, plasticity_dir)

In [ ]:

## Analysis parameters
threshold = 0.5
exclude = "Shaft Spine"
save = True

plasticity_data = {}
for key, value in coactivity_data.items():
    plasticity = Coactivity_Plasticity(
        value, threshold, exclude, vol_norm=False, save=save, save_path=plasticity_path
    )
    plasticity_data[key] = plasticity

### Visualize Coactivity Plasticity Data
Plot selected data and variables from individual datasets.
Use data generated above of load in a dataset

In [ ]:
# Select data to plot
load_data = False
initial_path = r"C:\Users\Jake\Desktop\Figures\grouped_data"
figure_path = os.path.join(initial_path, experiment_dir, plasticity_dir, "Early")
if not os.path.isdir(figure_path):
    os.makedirs(figure_path)
if load_data:
    load_name = "Early_dFoF_norm_0.5_coactivity_plasticity_data"
    plot_data = load_pickle([load_name], plasticity_path)[0]
else:
    plot_data = plasticity_data["Early"]


In [ ]:
# Print out variable names
for key in plot_data.__dict__.keys():
    print(key)

In [ ]:
print(sum(plot_data.enlarged_spines))
print(sum(plot_data.shrunken_spines))
print(sum(plot_data.stable_spines))

In [ ]:
# Correlation Plots
plot_data.plot_volume_correlation(
    variable_name="spine_activity_rate",
    volume_type="relative_volume",
    CI=95,
    ytitle="Spine Calcium dF/F",
    xlim=None,
    ylim=None,
    face_color="mediumblue",
    edge_color="white",
    edge_width=1,
    s_alpha=0.4,
    line_color="mediumblue",
    line_width=1,
    log_trans=True,
    save=False,
    save_path=figure_path
)

In [ ]:
# Group mean activity plots
plot_data.plot_group_scatter_plot(
    variable_name="conj_spine_coactive_calcium",
    group_type="plastic_spines",
    mean_type="mean",
    err_type="sem",
    figsize=(3,4),
    ytitle="Relative Coactivity Rate",
    ylim=None,
    s_colors=["darkorange", "forestgreen", "silver"],
    s_size=5,
    s_alpha=0.7,
    b_colors=["darkorange", "forestgreen", "silver"],
    b_edgecolors = "black",
    b_err_colors="black",
    b_width=0.7,
    b_linewidth=0,
    b_alpha=0.3,
    test_type="nonparametric",
    test_method="holm-sidak",
    save=False,
    save_path=figure_path,
)

In [ ]:

plot_data.plot_multi_group_scatter_plots(
    variable_name="conj_spine_coactive_calcium",
    group_type="movement_spines",
    subgroup_type="plastic_spines",
    mean_type="mean",
    err_type="sem",
    figsize=(7,4),
    ytitle="Fraction of Dendritic Events",
    ylim=None,
    s_colors=["darkorange", "forestgreen", "silver"],
    s_alpha=0.8,
    s_size=5,
    b_colors=["darkorange", "forestgreen", "silver"],
    b_edgecolors="black",
    b_err_colors="black",
    b_width=0.2,
    b_linewidth=0,
    b_alpha=0.3,
    test_method="sidak",
    save=False,
    save_path=figure_path,
)

In [ ]:
plot_data.plot_group_spine_mean_traces(
    group_type="plastic_spines",
    trace_type="conj_spine_coactive_calcium_traces",
    exclude=[],
    avlines=(30,90),
    ahlines=[0.10,0.11,0.12],
    figsize=(7.5,5),
    colors=["darkorange", "forestgreen", "silver"], 
    ylim=(0, 0.15),
    test_method="fdr_bh",
    save=False,
    save_path=figure_path
)

In [ ]:
plot_data.plot_spine_coactivity_distance(
    variable_name="relative_distance_movement_corr",
    group_type="plastic_spines",
    figsize=(5,5),
    colors=["darkorange", "forestgreen", "silver"],
    ylim=None,
    ytitle="Local Normalized Coactivity Rate",
    test_method="holm-sidak",
    save=False,
    save_path=figure_path,
)

In [ ]:
plot_data.plot_histogram(
    variable="conj_relative_onset",
    bins=50,
    max_lim=None,
    avlines=(0,0),
    figsize=(5,5),
    color="blue",
    alpha=0.4,
    save=False,
    save_path=figure_path
)

In [ ]:
plot_data.plot_group_mean_heatmaps(
    trace_type="conj_spine_coactive_calcium_traces",
    group_type="plastic_spines",
    figsize=(10,5),
    hmap_range=(0,0.2),
    center=None,
    sorted="peak",
    normalize=False,
    cmap="plasma",
    save=False,
    save_path=figure_path
)

In [ ]:
len(plot_data.ind_distance_coactivity_rate)

In [ ]:
original = np.log10(a.relative_volumes)
original = a.relative_volumes
norm = b.relative_volumes

In [ ]:
plot_sns_scatter_correlation(
    original,
    norm,
    CI=95,
    title="Relative Volume Comparison",
    xtitle="Original Relative Volume",
    ytitle="Normalized Relative Volume",
    marker_size=4,
    
)

In [ ]:
a = [1,0,0,0,1,1,0,0,1,1,1,1,0,1]
print(len(a))
print(sum(a))

In [ ]:
np.nanmean(a)